
---

### 🧪 Script: Aurum Backtest Engine (Verified)

> **Fase:** Validação / Simulação
> **Arquivo Alvo:** N/A (Gera Relatórios e Gráficos em Tempo Real)

Este script executa uma simulação histórica rigorosa (*Backtesting*) utilizando o framework **Backtrader**. Ele consome o histórico de scores gerado pelo motor de inteligência e simula o comportamento de uma carteira que compra sistematicamente as empresas com maior "Aurum Quality Score".

### 🎯 Objetivos

1. **Prova de Conceito:** Verificar se a estratégia de "Alta Qualidade + Bom Sentimento" supera o mercado ou protege o capital.
2. **Teste de Estresse:** Medir o risco máximo (`Max Drawdown`) em períodos de crise.
3. **Validação de Execução:** Garantir que a lógica de rebalanceamento mensal e custos de transação seja realista.

### 🏗️ Arquitetura de Simulação (Flowchart)

O fluxo simula a passagem do tempo, mês a mês, tomando decisões de compra e venda baseadas puramente no Score.

```mermaid
flowchart TD
    %% --- INPUT ---
    Input[("<b>Input: Scored History</b><br/>aurum_scored_history.parquet")]

    %% --- PREPARAÇÃO ---
    subgraph Prep ["1. Preparação de Dados"]
        Load["<b>Carregamento & Tipagem</b><br/>Conversão Float/Date"]
        Sanity["<b>Filtro de Sanidade</b><br/>Banir Tickers com Splits Errados<br/>(>100% ou <-80%)"]
        TimeFilter["<b>Janela Temporal</b><br/>Ex: 2023-2025"]
    end

    %% --- MOTOR BACKTRADER ---
    subgraph Engine ["2. Backtrader Engine (Cerebro)"]
        Feed["<b>Data Feeds</b><br/>Injeção de 70+ Ativos"]
        
        subgraph Strategy ["Lógica da Estratégia"]
            Signal["<b>Sinal (Mensal)</b><br/>Ler Aurum Score"]
            Rank["<b>Ranking</b><br/>Ordenar Top 10"]
            Rebal["<b>Rebalanceamento</b><br/>Vender Perdedores<br/>Comprar Vencedores"]
        end
        
        Broker["<b>Broker Simulado</b><br/>Caixa: R$ 100k<br/>Comissão: 0.05%"]
    end

    %% --- RESULTADOS ---
    subgraph Output ["3. Relatórios"]
        Metrics["<b>Métricas de Risco</b><br/>Sharpe, Drawdown, ROI"]
        Plot["<b>Gráfico de Patrimônio</b><br/>Curva de Capital"]
    end

    %% --- LIGAÇÕES ---
    Input --> Load --> Sanity --> TimeFilter
    TimeFilter --> Feed --> Engine
    Feed --> Signal --> Rank --> Rebal
    Rebal --> Broker --> Metrics & Plot

```

### ⚙️ Detalhes da Estratégia (`AurumVerifiedStrategy`)

A estratégia é classificada como **Long-Only Systematic Ranking** (Comprada, Sistemática e baseada em Ranking).

#### 1. Regra de Entrada (Buy)

* **Frequência:** Mensal (`rebalance_days = 1`).
* **Critério:** Selecionar os **Top N** ativos (ex: Top 10) com o maior `aurum_quality_score`.
* **Alocação:** Pesos iguais (`Equal Weight`). Se Top 10, compra 9.5% do capital em cada (deixando margem de segurança para taxas).

#### 2. Regra de Saída (Sell)

* Se um ativo sai do Top N (porque seu score caiu ou outros subiram), ele é **vendido integralmente** no próximo rebalanceamento.
* Isso garante que a carteira sempre reflita a "Nata" da qualidade, descartando empresas que pioraram seus fundamentos ou tiveram notícias ruins.

#### 3. Gestão de Risco

* **Diversificação:** O limite de concentração é definido pelo `top_n`.
* **Filtro de Sanidade:** Antes de começar, o script remove tickers com variações de preço irreais (ex: 900% em um mês) que poderiam distorcer o resultado final com lucros falsos.

## 📂 Outputs (Saídas)

Diferente dos scripts anteriores, este não gera um arquivo `.parquet`, mas sim **Informação Decisória**:

1. **Console Log:** Um diário detalhado de cada operação (Data, Compra, Venda, Caixa).
2. **Métricas Financeiras:**
* **ROI (Retorno):** Lucro total percentual.
* **Sharpe Ratio:** Retorno ajustado ao risco (Eficiência).
* **Max Drawdown:** A pior queda enfrentada (Segurança).


3. **Gráfico (Plot):** Visualização da curva de patrimônio ao longo do tempo.

### 🚀 Como Executar

Este é o "Grand Finale" do projeto.

```bash
python src/step_09_backtest_final_verified.py

```

### 📋 Exemplo de Log de Execução

```text
2025-06-30 | TOP 10 ESCOLHIDOS: ['WEGE3.SA', 'PSSA3.SA', 'ITUB4.SA'...]
2025-07-31 | VENDENDO VALE3.SA (Saiu do Portfolio)
...
🏆 RESULTADO FINAL - AURUM
Saldo Final:    R$ 106,496.62
Retorno Total:  6.50%
Max Drawdown:   3.17%

```

---

In [7]:
import backtrader as bt
import pandas as pd
import os
import logging
import numpy as np

# Configuração de Log
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)


In [8]:
# --- CONFIGURAÇÃO GERAL ---
DATA_FILE = "../data/aurum_final_scores/aurum_scored_history.parquet"
INITIAL_CASH = 100000.0
COMISSAO = 0.0005

In [9]:

# --- 1. CLASSE DE DADOS (IMPORTAÇÃO SEGURA) ---
class AurumVerifiedData(bt.feeds.PandasData):
    """
    Classe de dados customizada para ler o arquivo Aurum Scored.
    Removemos colunas de texto (grades) para evitar erros no Backtrader.
    """
    lines = ('aurum_score',) 
    
    params = (
        ('datetime', None),
        ('open', 'Adj Close'),
        ('high', 'Adj Close'),
        ('low', 'Adj Close'),
        ('close', 'Adj Close'),
        ('volume', -1),       
        ('openinterest', -1),
        ('aurum_score', 'aurum_quality_score'),
    )


In [10]:
class AurumVerifiedStrategy(bt.Strategy):
    params = (
        ('top_n', 5),        
        ('rebalance_days', 1),   
    )

    def log(self, txt, dt=None):
        """Função auxiliar para imprimir com a data da simulação"""
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} | {txt}')

    def __init__(self):
        self.timer = 0
        self.inds = {}
        for d in self.datas:
            self.inds[d] = d.aurum_score

    def next(self):
        self.timer += 1
        if self.timer < self.params.rebalance_days:
            return
        self.timer = 0
        
        self.log(f'--- REBALANCEAMENTO (Cash: {self.broker.get_cash():.2f}) ---')

        candidates = []
        for d in self.datas:
            if len(d) > 0 and d.aurum_score[0] > 0 and d.close[0] > 0:
                candidates.append((d, d.aurum_score[0]))
        
        candidates.sort(key=lambda x: x[1], reverse=True)
        
        top_picks = [item[0] for item in candidates[:self.params.top_n]]
        
        pick_names = [d._name for d in top_picks]
        self.log(f'TOP {self.params.top_n} ESCOLHIDOS: {pick_names}')
        
        for d in self.datas:
            pos = self.getposition(d).size
            if pos > 0:
                if d not in top_picks:
                    self.log(f'VENDENDO {d._name} (Saiu do Portfolio)')
                    self.close(d)
        
        if len(top_picks) > 0:
            target_pct = 0.95 / len(top_picks)
            for d in top_picks:
                self.order_target_percent(d, target=target_pct)

In [11]:
def filtrar_tickers_problematicos(df, limite_alta=1.0, limite_baixa=-0.8):
    print("🧹 Iniciando Filtro de Sanidade...")
    tickers_antes = df['ticker'].nunique()
    
    df['retorno_check'] = df.groupby('ticker')['Adj Close'].pct_change()
    
    bad_tickers = df[
        (df['retorno_check'] > limite_alta) | 
        (df['retorno_check'] < limite_baixa)
    ]['ticker'].unique()
    
    if len(bad_tickers) > 0:
        print(f"🚫 BANINDO {len(bad_tickers)} tickers problemáticos: {list(bad_tickers)}")
        df = df[~df['ticker'].isin(bad_tickers)].copy()
    
    print(f"✅ Tickers restantes: {df['ticker'].nunique()}")
    return df

In [12]:
def run_verified_backtest():
    print("--- 🦁 BACKTEST FINAL VERIFICADO: PROJETO AURUM ---")
    
    if not os.path.exists(DATA_FILE):
        print(f"❌ Erro Crítico: Arquivo {DATA_FILE} não encontrado.")
        return

    cerebro = bt.Cerebro()

    try:
        df = pd.read_parquet(DATA_FILE)
        
        df['date'] = pd.to_datetime(df['date'])
        df['Adj Close'] = pd.to_numeric(df['Adj Close'], errors='coerce')
        df['aurum_quality_score'] = pd.to_numeric(df['aurum_quality_score'], errors='coerce')
        
        df = df.dropna(subset=['date', 'ticker', 'Adj Close', 'aurum_quality_score'])
        df = df.sort_values('date')
        
    except Exception as e:
        print(f"❌ Erro ao ler dataset: {e}")
        return

    df = filtrar_tickers_problematicos(df)
    
    start_date = '2023-01-01'
    end_date = '2025-12-31'
    
    mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    df_final = df.loc[mask]
    
    if df_final.empty:
        print("⚠️ O filtro de data retornou vazio. Verifique as datas do arquivo.")
        return

    tickers = df_final['ticker'].unique()
    print(f"\n📈 Carregando histórico de {len(tickers)} ativos ({start_date} a {end_date})...")
    
    for t in tickers:
        df_t = df_final[df_final['ticker'] == t].copy()
        df_t = df_t.set_index('date').sort_index()
        
        if len(df_t) < 6: continue
        
        data = AurumVerifiedData(dataname=df_t, name=t)
        cerebro.adddata(data)

    cerebro.addstrategy(AurumVerifiedStrategy, top_n=10)
    cerebro.broker.setcash(INITIAL_CASH)
    cerebro.broker.setcommission(commission=COMISSAO)
    
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.11, timeframe=bt.TimeFrame.Months)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='dd')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='ret')

    print(f"\n💰 Saldo Inicial: R$ {INITIAL_CASH:,.2f}")
    results = cerebro.run()
    strat = results[0]
    
    val = cerebro.broker.getvalue()
    roi = ((val - INITIAL_CASH) / INITIAL_CASH) * 100
    
    print("\n" + "="*40)
    print(f"🏆 RESULTADO FINAL - AURUM")
    print("="*40)
    print(f"Saldo Final:    R$ {val:,.2f}")
    print(f"Lucro Líquido:  R$ {val - INITIAL_CASH:,.2f}")
    print(f"Retorno Total:  {roi:.2f}%")
    
    try:
        s_val = strat.analyzers.sharpe.get_analysis()['sharperatio']
        dd_val = strat.analyzers.dd.get_analysis()['max']['drawdown']
        
        s_str = f"{s_val:.3f}" if s_val is not None else "N/A"
        print(f"Sharpe Ratio:   {s_str}")
        print(f"Max Drawdown:   {dd_val:.2f}%")
    except:
        print("Métricas de Risco não puderam ser calculadas.")

    print("\n📉 Gerando gráfico de patrimônio...")
    try:
        cerebro.plot(style='line', iplot=False, volume=False)
    except Exception as e:
        print(f"⚠️ Aviso: Não foi possível plotar o gráfico. ({e})")

if __name__ == "__main__":
    run_verified_backtest()

--- 🦁 BACKTEST FINAL VERIFICADO: PROJETO AURUM ---
🧹 Iniciando Filtro de Sanidade...
🚫 BANINDO 19 tickers problemáticos: ['COGN3.SA', 'PRIO3.SA', 'GOAU4.SA', 'USIM5.SA', 'FLRY3.SA', 'PCAR3.SA', 'MGLU3.SA', 'AZZA3.SA', 'SLCE3.SA', 'PSSA3.SA', 'CPLE3.SA', 'IRBR3.SA', 'LREN3.SA', 'CSAN3.SA', 'CPLE5.SA', 'UGPA3.SA', 'TOTS3.SA', 'CEAB3.SA', 'TEND3.SA']
✅ Tickers restantes: 75

📈 Carregando histórico de 75 ativos (2023-01-01 a 2025-12-31)...

💰 Saldo Inicial: R$ 100,000.00
2025-06-30 | --- REBALANCEAMENTO (Cash: 100000.00) ---
2025-06-30 | TOP 10 ESCOLHIDOS: ['MULT3.SA', 'ALOS3.SA', 'BRKM5.SA', 'CXSE3.SA', 'SUZB3.SA', 'IGTI11.SA', 'TIMS3.SA', 'CURY3.SA', 'VALE3.SA', 'VIVA3.SA']
2025-07-01 | --- REBALANCEAMENTO (Cash: 24079.17) ---
2025-07-01 | TOP 10 ESCOLHIDOS: ['MULT3.SA', 'ALOS3.SA', 'BRKM5.SA', 'CXSE3.SA', 'SUZB3.SA', 'IGTI11.SA', 'TIMS3.SA', 'CURY3.SA', 'VALE3.SA', 'VIVA3.SA']
2025-07-31 | --- REBALANCEAMENTO (Cash: 24079.17) ---
2025-07-31 | TOP 10 ESCOLHIDOS: ['MULT3.SA', 'ALOS3.SA', 